# Project Monte Carlo, Asian Options

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import norm
random.seed(101)

### Simulation of the path of the underlying asset and the path of the brownian motion

In [2]:
def black_scholes_path (S0,T,N,r,sigma):
    """
    Simulate the underlying's path
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
        
    Return :
    - S : List representing the simulated path of the underlying asset
    """
    h = T / N

    # Generate random normal values
    z = np.random.normal(0, 1, N)

    # Initialize arrays
    S = np.zeros(N + 1)
    S[0] = S0

    # Generate the path
    for i in range(1, N+1):
        drift = (r - 0.5 * sigma**2) * h
        diffusion = sigma * z[i - 1]* np.sqrt(h)
        S[i] = S[i - 1] * np.exp(drift + diffusion)

    return S



In [3]:
def brownian_motion(T,N):
    """
    Compute the brownian's path
    
    Parameters :
    - T : maturity
    - N : number of time steps
    
    Return :
    - W, array containing the path of the brownian motion
    
    """
    h = T/N
    # Generate random increments
    increments = np.random.normal(0, np.sqrt(h), N)
    
    # Calculate cumulative sum to get the path
    W = np.cumsum(increments)
    
    # Add the starting point (0) to the path
    W = np.insert(W, 0, 0)
    
    return W

In [4]:
def black_scholes_bis (S0,T,N,r,sigma,W) :
    """
    Simulate the underlying's path
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - W : path of the brownian motion
        
    Return :
    - S : List representing the simulated path of the underlying asset
    """
    S = [S0]
    h = T/N
    for i in range(1,N+1):
        drift = (r - 0.5 * sigma**2) * h
        diffusion = sigma * (W[i]-W[i-1])
        S.append(S[i-1]*np.exp(drift+diffusion))
    return S

### Payoff functions for Asian options

In [5]:
def call_fixed_strike (int_S,K) :
    """
    Give the payoff of a call with fixed strike
    
    Parameters :
    - int_S : estimation of 1/t*(integral of 0 to t of Su du)
    - K : fixed strike
    
    Return :
    Payoff of a call option with fixed strike
    """
    if int_S-K > 0 :
        return int_S-K
    else :
        return 0
    
def put_fixed_strike (int_S,K) :
    
    """
    Give the payoff of a put with fixed strike
    
    Parameters :
    - int_S : estimation of 1/t*(integral of 0 to t of Su du)
    - K : fixed strike
    
    Return :
    Payoff of a put option with fixed strike
    """
    if K-int_S > 0 :
        return (K-int_S)
    else :
        return 0
    
def call_float_strike (int_S,S)  :
    """
    Give the payoff of a call with floating strike
    
    Parameters :
    - int_S : estimation of 1/t*(integral of 0 to t of Su du)
    - S : value of the underlying at maturity
    """
    if int_S-S > 0 :
        return S- int_S
    else :
        return 0   
    
def put_float_strike (int_S,S) :
    """
    Give the payoff of a put with floating strike
    
    Parameters :
    - int_S : estimation of 1/t*(integral of 0 to t of Su du)
    - S : value of the underlying at maturity
    """
    if S-int_S > 0 :
        return int_S - S
    else :
        return 0  

### Pricing with Monte Carlo - Riemann scheme

In [6]:
def Riemann_scheme (S0,T,N,r,sigma) :
    """
    Compute an estimation of 1/T*(integral of 0 to T of Su du) with the Riemann scheme
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    
    Return :
    - A list containing an estimation of 1/T*(integral of 0 to T of Su du) with the Riemann scheme and the
    value of the underlying at time T
    """
    h = T/N #Step size
    
    # Compute the path of the underlying
    BS_path = black_scholes_path (S0,T,N,r,sigma)
    #W = brownian_motion(T,N)
    #BS_path = black_scholes_bis (S0,T,N,r,sigma,W)
    
    # Compute the Riemann scheme
    YT = h*sum(BS_path[:N])/T
    
    return [YT,BS_path[N]]
    
    

In [7]:
def MC_price_Riemann (M,S0,T,N,r,sigma,K,option) :
    """
    Compute the price of an Asian option with a Monte Carlo method using the Riemann scheme
    
    Parameters:
    - M : number of monte carlo simulations
    - S0 : initial price of the underlying
    - T : maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - K : strike
    - option : type of option (type "Call_fixed" for a fixed call, "Put_fixed" for a fixed put, "Call_float" for
    a floating call, "Put_float" for a floating put)
    
    Return :
    A list containing the price of the option, the variance of the sample, the error, the lower bound for the
    confidence interval and the upper bound for the confiance interval
    
    """
    
    payoff = []
    
    for j in range(0,M) :
    
        Riemann = Riemann_scheme(S0,T,N,r,sigma)
        int_S = Riemann[0]
    
        if option == "Call_fixed" :
            payoff.append(call_fixed_strike (int_S,K))
            
        elif option == "Call_float" :
            S = Riemann[1]
            payoff.append(call_float_strike(int_S,S))
            
        elif option == "Put_fixed" :
            payoff.append(put_fixed_strike(int_S,K))
            
        elif option == "Put_float" :
            S = Riemann[1]
            payoff.append(put_float_strike(int_S,S))
            
        else :
            print("Not valid option parameter")
            return None
        
    price = np.exp(-r*T)*np.mean(payoff)
    standard_dev = np.std(np.array(payoff)*np.exp(-r*T))
    error = standard_dev/np.sqrt(M)
    CI_down = price-1.96*error
    CI_up = price+1.96*error
        
    return [price,standard_dev**2,error,CI_down,CI_up]
       

In [10]:
# Test of the algorithm using the results presented in the course of Bruno Bouchard "Monte Carlo for finance"

r = 0.1
sigmas = [0.05,0.2,0.3]
T = 1
S0 = 100
K = 100
N = 50
M = 10**5
option = "Call_fixed"

# Display of the results
print("Interest rate :",r)
print("Maturity :",T)
print("Initial price :",S0)
print("Strike price :",K)
print("Number of time steps :",N)
print("Number of Monte Carlo simulation :",M)
print("Kind of option :",option)
print()


for sigma in sigmas :
    print("Volatility :",sigma)
    result = MC_price_Riemann (M,S0,T,N,r,sigma,K,option)
    print(f"Price of the option : ${result[0]:,.2f}")
    print("Error :",result[2])
    print("Variance :",result[1])
    print("Confidence Interval :","[",round(result[3],2),",",round(result[4],2),"]")
    print()



Interest rate : 0.1
Maturity : 1
Initial price : 100
Strike price : 100
Number of time steps : 50
Number of Monte Carlo simulation : 100000
Kind of option : Call_fixed

Volatility : 0.05
Price of the option : $4.64
Error : 0.008371437216770276
Variance : 7.008096107432649
Confidence Interval : [ 4.63 , 4.66 ]

Volatility : 0.2
Price of the option : $6.92
Error : 0.02652961666922407
Variance : 70.38205606159717
Confidence Interval : [ 6.87 , 6.97 ]

Volatility : 0.3
Price of the option : $8.93
Error : 0.0391818438564584
Variance : 153.5216887991887
Confidence Interval : [ 8.85 , 9.0 ]



## Pricing Monte Carlo - Trapezoidal scheme

In [8]:
def Trapezoidal_scheme (S0,T,N,r,sigma) :
    """
    Compute an estimation of 1/T*(integral of 0 to T of Su du) with the Trapezoidal scheme
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    
    Return :
    - A list containing an estimation of 1/T*(integral of 0 to T of Su du) with the Trapezoidal scheme and the
    value of the underlying at time T
    """
    h = T/N #Step size
    
    # Compute the path of the underlying
    BS_path = black_scholes_path (S0,T,N,r,sigma)
    #W = brownian_motion(T,N)
    #BS_path = black_scholes_bis (S0,T,N,r,sigma,W)
    path = np.array(BS_path[:N]) # path used to compute the estimator
    
    
    # Computation of Trapezoidal scheme
    W_2 = np.random.normal(0,np.sqrt(T/N),N)
    YT= h/T*sum(path*(1 + (0.5*r*h) + (sigma*0.5*W_2)))
    
    return [YT,BS_path[N]]


In [9]:
def MC_price_Trapezoidal (M,S0,T,N,r,sigma,K,option) :
    """
    Compute the price of an Asian option with a Monte Carlo method using the Trapezoidal scheme
    
    Parameters:
    - M : number of monte carlo simulations
    - S0 : initial price of the underlying
    - T : maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - K : strike
    - option : type of option (type "Call_fixed" for a fixed call, "Put_fixed" for a fixed put, "Call_float" for
    a floating call, "Put_float" for a floating put)
    
    Return :
    A list containing the price of the option, the variance of the sample, the error, the lower bound for the
    confidence interval and the upper bound for the confiance interval
    
    """
    
    payoff = []
    
    for j in range(0,M) :
    
        trap = Trapezoidal_scheme(S0,T,N,r,sigma)
        int_S = trap[0]
    
        if option == "Call_fixed" :
            payoff.append(call_fixed_strike (int_S,K))
            
        elif option == "Call_float" :
            S = trap[1]
            payoff.append(call_float_strike(int_S,S))
            
        elif option == "Put_fixed" :
            payoff.append(put_fixed_strike(int_S,K))
            
        elif option == "Put_float" :
            S = trap[1]
            payoff.append(put_float_strike(int_S,S))
            
        else :
            print("Not valid option parameter, try again")
            return None
        
    price = np.exp(-r*T)*np.mean(payoff)
    standard_dev = np.std(np.array(payoff)*np.exp(-r*T))
    error = standard_dev/np.sqrt(M)
    CI_down = price-1.96*error
    CI_up = price+1.96*error
        
    return [price,standard_dev**2,error,CI_down,CI_up]


In [25]:
# Test of the algorithm using the results presented in the course of Bruno Bouchard "Monte Carlo for finance"

r = 0.1
sigmas = [0.05,0.2,0.3]
T = 1
S0 = 100
K = 100
N = 50
M = 10**5
option = "Call_fixed"

# Display of the results
print("Interest rate :",r)
print("Maturity :",T)
print("Initial price :",S0)
print("Strike price :",K)
print("Number of time steps :",N)
print("Number of Monte Carlo simulation :",M)
print("Kind of option :",option)
print()


for sigma in sigmas :
    print("Volatility :",sigma)
    result = MC_price_Trapezoidal (M,S0,T,N,r,sigma,K,option)
    print(f"Price of the option : ${result[0]:,.2f}")
    print("Error :",result[2])
    print("Variance :",result[1])
    print("Confidence Interval :","[",round(result[3],2),",",round(result[4],2),"]")
    print()


Interest rate : 0.1
Maturity : 1
Initial price : 100
Strike price : 100
Number of time steps : 50
Number of Monte Carlo simulation : 100000
Kind of option : Call_fixed

Volatility : 0.05
Price of the option : $4.73
Error : 0.008380429398457638
Variance : 7.0231596902533076
Confidence Interval : [ 4.71 , 4.74 ]

Volatility : 0.2
Price of the option : $6.98
Error : 0.02664678956883523
Variance : 71.0051394325786
Confidence Interval : [ 6.93 , 7.03 ]

Volatility : 0.3
Price of the option : $8.93
Error : 0.03894510939073694
Variance : 151.6721545456467
Confidence Interval : [ 8.85 , 9.0 ]



## Pricing Monte Carlo - Black Scholes scheme 

In [10]:
def BS_scheme (S0,T,N,r,sigma) :
    """
    Compute an estimation of 1/T*(integral of 0 to T of Su du) with the Black-Scholes scheme
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    
    Return :
    - A list containing an estimation of 1/T*(integral of 0 to T of Su du) with the Black-Scholes scheme and the
    value of the underlying at time T
    """    
    h = T/N
    
    W = brownian_motion(T,N)
    S = black_scholes_bis(S0,T,N,r,sigma,W)
    path = np.array(S[:N])
    rand = []
    
    for k in range (0,N) :
        mean = (0.5*h*W[k]) + (0.5*h*W[k+1])
        rand.append(np.random.normal(mean,np.sqrt((1/12)*(h**3)))-h*W[k])
        
    rand = np.array(rand)
    YT= 1/T*sum(path*(h + (0.5*r*(h**2)) + (sigma*rand)))
    
    return [YT,S[N]]
    

In [11]:
def MC_price_BS (M,S0,T,N,r,sigma,K,option) :
    """
    Compute the price of an Asian option with a Monte Carlo method using the Black-Scholes scheme
    
    Parameters:
    - M : number of monte carlo simulations
    - S0 : initial price of the underlying
    - T : maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - K : strike
    - option : type of option (type "Call_fixed" for a fixed call, "Put_fixed" for a fixed put, "Call_float" for
    a floating call, "Put_float" for a floating put)
    
    Return :
    A list containing the price of the option, the variance of the sample, the error, the lower bound for the
    confidence interval and the upper bound for the confiance interval
    
    """
    
    payoff = []
    
    for j in range(0,M) :
    
        BS = BS_scheme(S0,T,N,r,sigma)
        int_S = BS[0]
    
        if option == "Call_fixed" :
            payoff.append(call_fixed_strike (int_S,K))
            
        elif option == "Call_float" :
            S = BS[1]
            payoff.append(call_float_strike(int_S,S))
            
        elif option == "Put_fixed" :
            payoff.append(put_fixed_strike(int_S,K))
            
        elif option == "Put_float" :
            S = BS[1]
            payoff.append(put_float_strike(int_S,S))
            
        else :
            print("Not valid option parameter, try again")
            return None
        
    price = np.exp(-r*T)*np.mean(payoff)
    standard_dev = np.std(np.array(payoff)*np.exp(-r*T))
    error = standard_dev/np.sqrt(M)
    CI_down = price-1.96*error
    CI_up = price+1.96*error
        
    return [price,standard_dev**2,error,CI_down,CI_up]


In [17]:
# Test of the algorithm using the results presented in the course of Bruno Bouchard "Monte Carlo for finance"

r = 0.1
sigmas = [0.05,0.2,0.3]
T = 1
S0 = 100
K = 100
N = 50
M = 10**5
option = "Call_fixed"

# Display of the results
print("Interest rate :",r)
print("Maturity :",T)
print("Initial price :",S0)
print("Strike price :",K)
print("Number of time steps :",N)
print("Number of Monte Carlo simulation :",M)
print("Kind of option :",option)
print()


for sigma in sigmas :
    print("Volatility :",sigma)
    result = MC_price_BS (M,S0,T,N,r,sigma,K,option)
    print(f"Price of the option : ${result[0]:,.2f}")
    print("Error :",result[2])
    print("Variance :",result[1])
    print("Confidence Interval :","[",round(result[3],2),",",round(result[4],2),"]")
    print()




Interest rate : 0.1
Maturity : 1
Initial price : 100
Strike price : 100
Number of time steps : 50
Number of Monte Carlo simulation : 100000
Kind of option : Call_fixed

Volatility : 0.05
Price of the option : $4.73
Error : 0.008530976345800275
Variance : 7.277755741260383
Confidence Interval : [ 4.71 , 4.75 ]

Volatility : 0.2
Price of the option : $7.05
Error : 0.027037499207630097
Variance : 73.10263634025982
Confidence Interval : [ 7.0 , 7.11 ]

Volatility : 0.3
Price of the option : $9.02
Error : 0.0397642141212907
Variance : 158.1192724683855
Confidence Interval : [ 8.95 , 9.1 ]



## Control variate method

### Expectation of $\mathbb{E}(e^{-rT}(Z_{T}-K)_{+})$

In [12]:
def expect_control_variable (S0,K,T,N,r,sigma) :
    
    sigma_tilde = sigma*np.sqrt(T/3)
    C1 = 0.5*T*(r-0.5*sigma**2) + ((T*sigma**2) / 6)
    C2 = (0.5*T*(r-0.5*sigma**2) - np.log(K/S0)) / sigma_tilde
    
    return np.exp(-r*T) * (S0*np.exp(C1)*norm.cdf(C2+sigma_tilde,0,1) - K*norm.cdf(C2,0,1))

### Control variate and Riemann scheme

In [24]:
def riemann_control (S0,T,N,r,sigma):

    """
    Compute an estimation of 1/T*(integral of 0 to T of Su du) and 1/T*(integral of 0 to T of Wu du) 
    with the Riemann scheme
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    
    Return :
    - A list containing an estimation of 1/T*(integral of 0 to T of Su du) with the Riemann scheme, an estimation
    Z_{T} as given in the pdf and the value of the underlying at time T
    """
    h = T/N #Step size
    
    # Compute the path of the underlying
    W = brownian_motion(T,N)
    BS_path = black_scholes_bis (S0,T,N,r,sigma,W)
    
    # Compute the Riemann scheme
    YT = h*sum(BS_path[:N])/T
    H_est = h*sum(W[:N])/T
    ZT = S0*np.exp((r-0.5*sigma**2)*0.5*T + sigma*H_est)
    
    return [YT,ZT,BS_path[N]] 

In [25]:
def MC_price_control_riemann (M,S0,T,N,r,sigma,K,option) :
    """
    Compute the price of an Asian option with a Monte Carlo method using the Riemann scheme and the control variate
    method
    
    Parameters:
    - M : number of monte carlo simulations
    - S0 : initial price of the underlying
    - T : maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - K : strike
    - option : type of option (type "Call_fixed" for a fixed call, "Put_fixed" for a fixed put, "Call_float" for
    a floating call, "Put_float" for a floating put)
    
    Return :
    A list containing the price of the option, the variance of the sample, the error, the lower bound for the
    confidence interval and the upper bound for the confiance interval
    
    """
    
    payoff_YT = []
    payoff_ZT = []
    
    for j in range(0,M) :
    
        riem = riemann_control (S0,T,N,r,sigma)
        int_S = riem[0]
        ZT = riem[1]
        
    
        if option == "Call_fixed" :
            payoff_YT.append(call_fixed_strike (int_S,K))
            payoff_ZT.append(call_fixed_strike(ZT,K))
            
        elif option == "Call_float" :
            S = riem[2]
            payoff_YT.append(call_float_strike(int_S,S))
            payoff_ZT.append(call_float_strike(ZT,S))
            
        elif option == "Put_fixed" :
            payoff_YT.append(put_fixed_strike(int_S,K))
            payoff_ZT.append(put_fixed_strike(ZT,K))
            
        elif option == "Put_float" :
            S = riem[2]
            payoff_YT.append(put_float_strike(int_S,S))
            payoff_ZT.append(put_float_strike(ZT,S))
            
        else :
            print("Not valid option parameter, try again")
            return None
        
    price = (np.exp(-r*T)*np.mean(payoff_YT) - np.exp(-r*T)*np.mean(payoff_ZT) + expect_control_variable (S0,K,T,N,r,sigma))
    standard_dev = np.std(np.exp(-r*T)*(np.array(payoff_YT) - np.array(payoff_ZT)))
    error = standard_dev/np.sqrt(M)
    CI_down = price-1.96*error
    CI_up = price+1.96*error
        
    return [price,standard_dev**2,error,CI_down,CI_up]


In [26]:
# Test of the algorithm using the results presented in the course of Bruno Bouchard "Monte Carlo for finance"

r = 0.1
sigmas = [0.05,0.2,0.3]
T = 1
S0 = 100
K = 100
N = 50
M = 10**5
option = "Call_fixed"

# Display of the results
print("Interest rate :",r)
print("Maturity :",T)
print("Initial price :",S0)
print("Strike price :",K)
print("Number of time steps :",N)
print("Number of Monte Carlo simulation :",M)
print("Kind of option :",option)
print()


for sigma in sigmas :
    print("Volatility :",sigma)
    result = MC_price_control_riemann (M,S0,T,N,r,sigma,K,option)
    print(f"Price of the option : ${result[0]:,.2f}")
    print("Error :",result[2])
    print("Variance :",result[1])
    print("Confidence Interval :","[",round(result[3],3),",",round(result[4],3),"]")
    print()


Interest rate : 0.1
Maturity : 1
Initial price : 100
Strike price : 100
Number of time steps : 50
Number of Monte Carlo simulation : 100000
Kind of option : Call_fixed

Volatility : 0.05
Price of the option : $4.63
Error : 0.00014658368061693663
Variance : 0.0021486775423208092
Confidence Interval : [ 4.634 , 4.634 ]

Volatility : 0.2
Price of the option : $6.99
Error : 0.0011972711353438422
Variance : 0.14334581715275332
Confidence Interval : [ 6.988 , 6.992 ]

Volatility : 0.3
Price of the option : $9.02
Error : 0.0026817325081433343
Variance : 0.7191689245232741
Confidence Interval : [ 9.015 , 9.025 ]



###  Control variate and Trapezoidal scheme

In [16]:
def Trapezoidal_control_scheme (S0,T,N,r,sigma) :
    """
    Compute an estimation of 1/T*(integral of 0 to T of Su du) with the Trapezoidal scheme and an estimation of ZT
    with the trapezoidal scheme
    
    Parameters :
    - S0 : initial price of the uunderlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    
    Return :
    - A list containing an estimation of 1/T*(integral of 0 to T of Su du) and a estimation of ZT with the 
    Trapezoidal scheme and the value of the underlying at time T
    """
    h = T/N #Step size
    
    # Compute the path of the underlying
    #BS_path = black_scholes_path (S0,T,N,r,sigma)
    W = brownian_motion(T,N)
    BS_path = black_scholes_bis (S0,T,N,r,sigma,W)
    under_path = np.array(BS_path[:N]) # path used to compute the estimator
    
    # Computation of Trapezoidal scheme
    W_2 = np.random.normal(0,np.sqrt(T/N),N)
    YT = h/T*sum(under_path*(1 + (0.5*r*h) + (sigma*0.5*W_2)))
    
    H_est = 0
    for k in range(0,N):
        H_est += (W[k]+W[k+1])
    H_est = H_est*0.5*h/T
    
    
    ZT = S0*np.exp((r-0.5*sigma**2)*0.5*T + sigma*H_est)
    
    return [YT,ZT,BS_path[N]]


In [17]:
def MC_price_control_trapez (M,S0,T,N,r,sigma,K,option) :
    """
    Compute the price of an Asian option with a Monte Carlo method using the Trapezoidal scheme and the control 
    variate method
    
    Parameters:
    - M : number of monte carlo simulations
    - S0 : initial price of the underlying
    - T : maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - K : strike
    - option : type of option (type "Call_fixed" for a fixed call, "Put_fixed" for a fixed put, "Call_float" for
    a floating call, "Put_float" for a floating put)
    
    Return :
    A list containing the price of the option, the variance of the sample, the error, the lower bound for the
    confidence interval and the upper bound for the confiance interval
    
    """
    
    payoff_YT = []
    payoff_ZT = []
    
    for j in range(0,M) :
    
        trap = Trapezoidal_control_scheme (S0,T,N,r,sigma)
        int_S = trap[0]
        ZT = trap[1]
        
    
        if option == "Call_fixed" :
            payoff_YT.append(call_fixed_strike (int_S,K))
            payoff_ZT.append(call_fixed_strike(ZT,K))
            
        elif option == "Call_float" :
            S = trap[2]
            payoff_YT.append(call_float_strike(int_S,S))
            payoff_ZT.append(call_float_strike(ZT,S))
            
        elif option == "Put_fixed" :
            payoff_YT.append(put_fixed_strike(int_S,K))
            payoff_ZT.append(put_fixed_strike(ZT,K))
            
        elif option == "Put_float" :
            S = trap[2]
            payoff_YT.append(put_float_strike(int_S,S))
            payoff_ZT.append(put_float_strike(ZT,S))
            
        else :
            print("Not valid option parameter, try again")
            return None
        
    price = (np.exp(-r*T)*np.mean(payoff_YT) - np.exp(-r*T)*np.mean(payoff_ZT) + expect_control_variable (S0,K,T,N,r,sigma))
    standard_dev = np.std(np.exp(-r*T)*(np.array(payoff_YT) - np.array(payoff_ZT)))
    error = standard_dev/np.sqrt(M)
    CI_down = price-1.96*error
    CI_up = price+1.96*error
        
    return [price,standard_dev**2,error,CI_down,CI_up]


In [19]:
# Test of the algorithm using the results presented in the course of Bruno Bouchard "Monte Carlo for finance"

r = 0.1
sigmas = [0.05,0.2,0.3]
T = 1
S0 = 100
K = 100
N = 50
M = 10**5
option = "Call_fixed"

# Display of the results
print("Interest rate :",r)
print("Maturity :",T)
print("Initial price :",S0)
print("Strike price :",K)
print("Number of time steps :",N)
print("Number of Monte Carlo simulation :",M)
print("Kind of option :",option)
print()


for sigma in sigmas :
    print("Volatility :",sigma)
    result = MC_price_control_trapez (M,S0,T,N,r,sigma,K,option)
    print(f"Price of the option : ${result[0]:,.2f}")
    print("Error :",result[2])
    print("Variance :",result[1])
    print("Confidence Interval :","[",round(result[3],3),",",round(result[4],3),"]")
    print()



Interest rate : 0.1
Maturity : 1
Initial price : 100
Strike price : 100
Number of time steps : 50
Number of Monte Carlo simulation : 100000
Kind of option : Call_fixed

Volatility : 0.05
Price of the option : $4.72
Error : 0.00017104698337021845
Variance : 0.0029257070520051797
Confidence Interval : [ 4.72 , 4.721 ]

Volatility : 0.2
Price of the option : $6.98
Error : 0.0010909678696637097
Variance : 0.11902108926385734
Confidence Interval : [ 6.981 , 6.985 ]

Volatility : 0.3
Price of the option : $8.96
Error : 0.002339341715419113
Variance : 0.5472519661500039
Confidence Interval : [ 8.959 , 8.968 ]



### Control variate and Black-Scholes scheme

In [21]:
def BS_scheme_control (S0,T,N,r,sigma) :
    """
    Compute an estimation of 1/T*(integral of 0 to T of Su du) and ZT with the Black-Scholes scheme
    
    Parameters :
    - S0 : initial price of the underlying
    - T : time to maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    
    Return :
    - A list containing an estimation of 1/T*(integral of 0 to T of Su du) and ZT with the Black-Scholes scheme 
    and the value of the underlying at time T
    """    
    h = T/N
    
    W = brownian_motion(T,N)
    S = black_scholes_bis(S0,T,N,r,sigma,W)
    path = np.array(S[:N])
    rand_YT = []
    rand_ZT = []
    
    for k in range (0,N) :
        mean = (0.5*h*W[k]) + (0.5*h*W[k+1])
        temp = np.random.normal(mean,np.sqrt((1/12)*(h**3)))
        rand_YT.append(temp - h*W[k])
        rand_ZT.append(temp)
        
        
        
    rand_YT = np.array(rand_YT)
    rand_ZT = np.array(rand_ZT)
    YT= 1/T*sum(path*(h + (0.5*r*(h**2)) + (sigma*rand_YT)))
    H_est = 1/T*sum(rand_ZT)
    
    ZT = S0*np.exp((r-0.5*sigma**2)*0.5*T + sigma*H_est)
    
    return [YT,ZT,S[N]]
  

In [22]:
def MC_price_control_BS (M,S0,T,N,r,sigma,K,option) :
    """
    Compute the price of an Asian option with a Monte Carlo method using the BS scheme and the control variate 
    method
    
    Parameters:
    - M : number of monte carlo simulations
    - S0 : initial price of the underlying
    - T : maturity
    - N : number of time steps
    - r : interest rate
    - sigma : volatility
    - K : strike
    - option : type of option (type "Call_fixed" for a fixed call, "Put_fixed" for a fixed put, "Call_float" for
    a floating call, "Put_float" for a floating put)
    
    Return :
    A list containing the price of the option, the variance of the sample, the error, the lower bound for the
    confidence interval and the upper bound for the confiance interval
    
    """
    
    payoff_YT = []
    payoff_ZT = []
    
    for j in range(0,M) :
    
        BS = BS_scheme_control (S0,T,N,r,sigma)
        int_S = BS[0]
        ZT = BS[1]
        
    
        if option == "Call_fixed" :
            payoff_YT.append(call_fixed_strike (int_S,K))
            payoff_ZT.append(call_fixed_strike(ZT,K))
            
        elif option == "Call_float" :
            S = BS[2]
            payoff_YT.append(call_float_strike(int_S,S))
            payoff_ZT.append(call_float_strike(ZT,S))
            
        elif option == "Put_fixed" :
            payoff_YT.append(put_fixed_strike(int_S,K))
            payoff_ZT.append(put_fixed_strike(ZT,K))
            
        elif option == "Put_float" :
            S = BS[2]
            payoff_YT.append(put_float_strike(int_S,S))
            payoff_ZT.append(put_float_strike(ZT,S))
            
        else :
            print("Not valid option parameter, try again")
            return None
        
    price = (np.exp(-r*T)*np.mean(payoff_YT) - np.exp(-r*T)*np.mean(payoff_ZT) + expect_control_variable (S0,K,T,N,r,sigma))
    standard_dev = np.std(np.exp(-r*T)*(np.array(payoff_YT) - np.array(payoff_ZT)))
    error = standard_dev/np.sqrt(M)
    CI_down = price-1.96*error
    CI_up = price+1.96*error
        
    return [price,standard_dev**2,error,CI_down,CI_up]


In [23]:
# Test of the algorithm using the results presented in the course of Bruno Bouchard "Monte Carlo for finance"

r = 0.1
sigmas = [0.05,0.2,0.3]
T = 1
S0 = 100
K = 100
N = 50
M = 10**5
option = "Call_fixed"

# Display of the results
print("Interest rate :",r)
print("Maturity :",T)
print("Initial price :",S0)
print("Strike price :",K)
print("Number of time steps :",N)
print("Number of Monte Carlo simulation :",M)
print("Kind of option :",option)
print()


for sigma in sigmas :
    print("Volatility :",sigma)
    result = MC_price_control_BS (M,S0,T,N,r,sigma,K,option)
    print(f"Price of the option : ${result[0]:,.2f}")
    print("Error :",result[2])
    print("Variance :",result[1])
    print("Confidence Interval :","[",round(result[3],3),",",round(result[4],3),"]")
    print()



Interest rate : 0.1
Maturity : 1
Initial price : 100
Strike price : 100
Number of time steps : 50
Number of Monte Carlo simulation : 100000
Kind of option : Call_fixed

Volatility : 0.05
Price of the option : $4.72
Error : 0.0001569176658378094
Variance : 0.002462315385198642
Confidence Interval : [ 4.724 , 4.725 ]

Volatility : 0.2
Price of the option : $7.04
Error : 0.0012715227411311222
Variance : 0.1616770081213603
Confidence Interval : [ 7.039 , 7.044 ]

Volatility : 0.3
Price of the option : $9.05
Error : 0.002713901100718331
Variance : 0.7365259184480172
Confidence Interval : [ 9.049 , 9.06 ]

